## **Dataset and Model Files**

In [0]:
import os
os.mkdir('Seq2Seq')

In [0]:
from google.colab import files
files.upload() #upload kaggle.json

Mount google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Download and process TIMIT dataset

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download mfekadu/darpa-timit-acousticphonetic-continuous-speech
!unzip -q darpa-timit-acousticphonetic-continuous-speech.zip -d Timit_dataset

In [0]:
import shutil
import os
os.mkdir('Seq2Seq/TIMIT')
shutil.move('Timit_dataset/data/TRAIN','Seq2Seq/TIMIT')
shutil.move('Timit_dataset/data/TEST','Seq2Seq/TIMIT')
shutil.rmtree('Timit_dataset', ignore_errors=True)
os.remove('darpa-timit-acousticphonetic-continuous-speech.zip')

Move to deepspeech directory

In [0]:
import os
os.chdir('Seq2Seq')

Upload model files

In [0]:
from google.colab import files
files.upload()

Prepare TIMIT dataset

In [0]:
!python3 prepare_timit.py

## **Model Training**

In [0]:
from model import create_model, create_optimizer, create_model_checkpoint_cb, create_lr_scheduler_cb
from visualization import plot_accuracy, plot_loss
from BeamSearch import ctcBeamSearch
from dataset import create_dataset
from generator import generator
from text import decode
import constants as c

from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as k
from tensorflow.keras.models import Model
import numpy as np

In [0]:
X_train, X_test, y_train, y_test = create_dataset()

In [0]:
model = create_model()

model.summary()

plot_model(model, to_file='rnn.png')

optimizer = create_optimizer()

loss = {'ctc': lambda y_true, y_pred: y_pred}

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [0]:
history = model.fit(
    generator(X_train, y_train, c.batch_size),
    steps_per_epoch=int(np.ceil(X_train.shape[0]/c.batch_size)),
    epochs=50,
    validation_data=generator(X_test, y_test, c.batch_size),
    validation_steps=int(np.ceil(X_test.shape[0]/c.batch_size)),
    callbacks=[create_model_checkpoint_cb(), create_lr_scheduler_cb()],
    initial_epoch=20
    )

In [0]:
plot_accuracy(history)
plot_loss(history)

In [0]:
model.load_weights(c.checkpoint_filepath)
model.save_weights("model.h5")

## **Model Prediction**

In [0]:
sub_model = Model(inputs=model.get_layer('masking_layer').input, outputs=model.get_layer('output_layer').output)

for i in range(10):
    data = X_test[i]
    d = np.array([data])
    
    prediction=sub_model.predict(d)
    output = k.get_value(prediction)        
    path = ctcBeamSearch(output[0], ''.join(c.alphabet), None)

    print('true:', decode(y_test[i]))
    print('pred:', path)
    print()